In [ ]:
#imports
import pandas as pd; import numpy as np; import re; import matplotlib.pyplot as plt;

In [ ]:
#config
extended_output = True

#inputs
kar_file_path = "inputs/Data_D1_karyotype.tsv"
data_file_path = "inputs/SJALL003310_D3.tsv"

#outputs
csv_file_path = "coverage/coverage_with_x_and_median.csv"
tsv_file_path = "coverage/coverage_with_x_and_median.tsv"

#plot outputs
plot_path = "plots/converage_plot.png"
simple_plot_path = "plots/simple_converage_plot.png"


In [ ]:
#helper functions

#gets midpoint
def proc_Pos (x):
    return (np.min(x) + np.max(x))/2

#helps get the chromosome position
def extract_chromosome(arm):
    if arm.startswith('chr'):
        if arm == 'chrXp':
            return 1000
        elif arm == 'chrXq':
            return 1001
        elif arm == 'chrYp':
            return 1002
        elif arm == 'chrYq':
            return 1003
        else:
            match = re.match(r'chr(\d+)([pq])', arm)
            if match:
                number = int(match.group(1))
                arm_type = match.group(2)
                # Ensure numeric chromosomes are ordered first
                return number * 2 + (1 if arm_type == 'q' else 0)
    return -1  # Return -1 if not matched

#view the data frame in theta(n) time
def dataViewer(data):
    for i in range (1,22):
        print("new chromosome: "+str(i))
        print(data[data['arm']=="chr"+str(i)+"p"])
        print(data[data['arm']=="chr"+str(i)+"q"])
        print("\n")
        
        

def plot(data,path):
    plt.figure(figsize=(20, 3))
    plt.scatter(data['x'].values, data['y'].values, s=5, alpha=0.5, c='lightgreen')  # s parameter controls the size of the dots
    plt.title('Coverage Plot')
    plt.xlabel('position')
    plt.ylabel('log2(median/ref)')
    plt.grid(True)


    x_ticks = []
    x_labels = []
    previous_arm = None
    
    for i, (x, arm) in enumerate(zip(data['x'], data['arm'])):
        if arm != previous_arm:
            x_ticks.append(x)
            x_labels.append(arm)
            previous_arm = arm
    
    plt.xticks(ticks=x_ticks, labels=x_labels, rotation=90)
    
    # Show the plot
    plt.tight_layout()
    plt.savefig(path) 
    plt.show()
    
def simplePlot(data,path):
    plt.figure(figsize=(20, 3))
    data.plot.scatter("x", "y")
    plt.savefig(path) 
    plt.show()
        

In [ ]:
#get dataframes
kar_data = pd.read_csv(kar_file_path, sep="\t")

data_i = pd.read_csv(data_file_path, sep="\t")

In [ ]:
#drop data
data_filter = data_i[data_i['cv'] < 20]
print("After Filtering CV: "+str(data_filter.shape))

data = data_filter.dropna(subset=['lcv', 'Pos'])


if extended_output: dataViewer(data)

In [ ]:
# I forgot the keyboard command to remove a cell

In [ ]:
ref_arms = kar_data.loc[kar_data['clone'] == 'DIP', 'arm'].tolist()
tmp = data.loc[[(a in ref_arms) & (not ho) for a, ho in zip(data['arm'].tolist(), data['Houtlier'].tolist())]]
# what I called r
mlcv = tmp['lcv'].mean()
print("mlcv : "+str(mlcv))

In [ ]:
grdata = tmp.groupby(by=['arm', 'group_tr']).agg({'lcv': np.mean, 'Pos': proc_Pos, 'cv': len}).reset_index()

if extended_output: dataViewer(data)

In [ ]:
grdata['y'] = np.log(grdata['lcv'].values / mlcv)

grdata

In [ ]:
grdata['arm_order'] = grdata['arm'].apply(extract_chromosome)
grdata = grdata.sort_values(by=['arm_order', 'group_tr']).reset_index(drop=True)
grdata['x'] = range(len(grdata))

grdata

In [ ]:
#exports
csv_file_path = "coverage/coverage_with_x_and_median.csv"
grdata.to_csv(csv_file_path, index=False)

tsv_file_path = "coverage/coverage_with_x_and_median.tsv"
grdata.to_csv(tsv_file_path, sep='\t', index=False)

In [ ]:
#look
dataViewer(grdata)


In [ ]:
#plot
plot(grdata,plot_path)


In [ ]:
#simple plot
simplePlot(grdata,simple_plot_path)